In [ ]:
import sys
sys.path.append('/Users/admin/Desktop/tesi/Thesis/')
from skimage.filters import gaussian
from handcrafted_descriptors.grid_classification import utils
from handcrafted_descriptors.descriptor import binary_gabor_features as bgf
import sklearn.metrics as skmetrics
import argparse
import cv2
from sklearn.metrics import precision_recall_fscore_support
import numpy as np 
import os

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# import the necessary packages
from skimage import feature
import numpy as np

In [ ]:
species = ['Arthonia_radiata','Caloplaca_cerina','Candelariella_reflexa','Candelariella_xanthostigma','Chrysothrix_candelaris','Flavoparmelia_caperata','Gyalolechia_flavorubescens','Hyperphyscia_adglutinata'
        ,'Lecanora_argentata','Lecanora_chlarotera','Lecidella_elaeochroma','Melanelixia_glabratula'
        ,'Phaeophyscia_orbicularis','Physcia_biziana','Physconia_grisea','Ramalina_farinacea','Ramalina_fastigiata','Xanthomendoza_fallax','Xanthomendoza_fulva','flavoparmenia_soredians']


In [147]:
def calculate_max_value(numpoints):
    res = 1
    for i in range(1,numpoints):
        res = res + 2**(i)
    return res


def plot_and_save_knn(acc,prec,rec,direct):
    x1 = np.arange(1,51)
    plt.plot(x1, acc, label = "accuracy",color = 'r')
    plt.plot(x1,prec, label = 'precision')
    plt.plot(x1, rec, label = 'recall',color = 'g')
    plt.xlabel('number of neighbors')
    plt.ylabel('percentage (%)')
    plt.legend()
    plt.grid()
    plt.savefig(direct)
    plt.close()


In [ ]:

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius,meth = 'ror'):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
        self.meth = meth
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        if (self.meth == 'default'):
            
            lbp = feature.local_binary_pattern(image, self.numPoints,self.radius,method = 'default')
        else:
            lbp = feature.local_binary_pattern(image, self.numPoints,self.radius,method = 'ror')
        return lbp.ravel()
    

    


In [ ]:
def getPath_and_dict(path, train):
    lichens = os.listdir(path)
    dic_lic = {}
    res = []
    for i,l in enumerate(lichens):
        dic_lic[l] = i
        pth = os.path.join(path,l)
        images_of_specific_lichen = os.listdir(pth)
        for j,im in enumerate(images_of_specific_lichen):
            image = os.path.join(pth,im)
            res.append(image)
    
    
    if(train is True):
        np.random.shuffle(res)
    return res, dic_lic


def readimage(im):
    img = cv2.imread(im)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


In [ ]:
def lbp_extract_descriptors(path, numPoints, radius,meth = 'default',train = True):       
    desc = LocalBinaryPatterns(numPoints, radius, meth = meth)
    images_pth, label_dict = getPath_and_dict(path, train)
    descriptor_list = [] 
    train_labels = []
    image_count = len(images_pth)  
    for i,img_path in enumerate(images_pth):
        cl = img_path.split('/')[-2]
        class_index = cl
        train_labels.append(class_index)        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hist1 = desc.describe(img[:,:,0])
        hist2 = desc.describe(img[:,:,1])
        hist3 = desc.describe(img[:,:,2])
        lbp_total = np.concatenate([hist1,hist2,hist3])
        if meth == 'uniform':
            maximum = calculate_max_value(numPoints)
            (hist, _) = np.histogram(lbp_total.ravel(),bins=np.arange(0, maximum + 3),
                        range=(0, maximum + 2))
        else: 
            maximum = calculate_max_value(numPoints)
            (hist, _) = np.histogram(lbp_total.ravel(),bins=np.arange(0, maximum + 3),
                        range=(0, maximum + 2))
        hist = hist / np.sum(hist)
        descriptor_list.append(hist)     
    descriptors =np.array(descriptor_list)
    train_labels = np.array(train_labels)
    return descriptors, train_labels
    
    

In [103]:
def search_for_best_poly(training_feat, tr_lab, tst_features, tst_lab):
    param_grid = {'degree':[2,3,4,5,6,7],'C': [1e-3,1e-2, 1e-1, 1e0, 1e1, 1e2,1e3], 'gamma': [  0.00001,0.0001,0.001, 0.01, 0.1,1]}
    clf0 = GridSearchCV(SVC(kernel='degree'), param_grid)
    svc = clf0.fit(training_feat, tr_lab)
    print("Best estimator found by grid search : ", clf0.best_estimator_)
    y_pred = clf0.predict(tst_features)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, tst_lab))
    return clf0, skmetrics.accuracy_score(y_pred, tst_lab)

In [ ]:
def svcParamSelection(X, y, kernel, nfolds):
    Cs = [0.5, 0.1, 0.15, 0.2, 0.3]
    gammas = [0.1, 0.11, 0.095, 0.105]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel=kernel), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
def search_for_best_rbf(training_feat, tr_lab, tst_features, tst_lab):
    param_grid = {'C': [ 1e0, 1e1, 1e2,1e3], 'gamma': [   0.01, 0.1,1]}
    clf0 = GridSearchCV(SVC(kernel='rbf'), param_grid)
    svc = clf0.fit(training_feat, tr_lab)
    print("Best estimator found by grid search : ", clf0.best_estimator_)
    y_pred = clf0.predict(tst_features)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, tst_lab))
    return clf0,skmetrics.accuracy_score(y_pred, tst_lab)

In [ ]:
def create_and_save_confusion_matrix(tst_features, tst_lab, species, vw,director, name ):
    pred = svm.predict(tst_features)
    df = utils.evaluated_prediction(pred, tst_lab, species)
    cm = utils.build_confusion_matrix(df, pred, tst_lab,species)
    fig=plt.figure(figsize=(30, 15))
    utils.plot_confusion_matrix(cm,species,name,director,normalize=True,title='Confusion matrix')
    return pred



def calculate_and_plot_precision_recall(tst_lab, pred, species, directory, string):
    precision, recall, fbeta, support = precision_recall_fscore_support(tst_lab, pred)
    
    df = pd.DataFrame({"X":species, "precision":precision,"recall":recall,'f1score': fbeta})
    df.plot(x="X", y=["precision", "recall",'f1score'], kind="bar")
    plt.tight_layout()
    plt.savefig(directory +'precision_recall_class4class'+string+'.jpg')
    return np.mean(precision), np.mean(recall), np.mean(fbeta)


## MAIN

In [162]:
import pandas as pd
accuracy = []
precision = []
recall = []
for radius in [1,2,3,4,5,6,7,8,9,10]:
    print('start calculating descriptor')
    feat,lab = lbp_extract_descriptors('../../../data/train', 8, radius,meth = 'ror',train = True)    
    test_feat, test_lab = lbp_extract_descriptors('../../../data/valid',8, radius,meth = 'ror',train = False)
    print('end calculating descriptor')
    print('start svm')
    svm, acc =search_for_best_poly(feat, lab, test_feat, test_lab)
    print('end svm')
    #pred = svm.predict(test_feat)
    #df = evaluated_prediction(pred, test_lab, species)
    #cm = build_confusion_matrix(df, pred, test_lab,species)
    #fig=plt.figure(figsize=(30, 15))
    #utils.plot_confusion_matrix(cm,species,'confusion_matrix_' + str(radius) + '_8points','results_lbp/',normalize=True,title='Confusion matrix')
    #prec, rec, fbeta = calculate_and_plot_precision_recall(test_lab, pred, species, 'results_lbp/', 'precision_recall_' + str(radius) + '_8points')
    #precision.append(prec)
    #recall.append(rec)
    #accuracy.append(acc)

start calculating descriptor
end calculating descriptor
start svm


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

...

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 217, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py", line 276, in _dense_fit
    max_iter=self.max_iter, random_seed=random_seed)
  File "sklearn/svm/_libsvm.pyx", line 175, in sklearn.svm._libsvm.fit
ValueError: 'degree' is not in list

  FitFailedWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test part

ValueError: 'degree' is not in list

In [152]:
x1 = [1,2,3,4,5,6,7,8,9,10]
df = pd.DataFrame({"X":x1, "precision":precision,"recall":recall,'accuracy': accuracy})
df.plot(x="X", y=["precision", "recall",'accuracy'], kind="bar")
plt.grid()
plt.xlabel('density of the grid (in pixel)')
plt.ylabel('Percentate %')
plt.tight_layout()
plt.savefig('results_lbp/total8points_bar.jpg')

ValueError: arrays must all be same length

## MULTI-SCALE APPROACH

In [159]:
def multiscale_lbp_extract_descriptors(path, numPoints ,meth = 'default',train = True):       
    desc = [LocalBinaryPatterns(numPoints, 1, meth = meth), LocalBinaryPatterns(numPoints, 2, meth = meth),LocalBinaryPatterns(numPoints, 3, meth = meth),LocalBinaryPatterns(numPoints, 4, meth = meth),LocalBinaryPatterns(numPoints, 5, meth = meth)]
    images_pth, label_dict = getPath_and_dict(path, train)
    descriptor_list = [] 
    train_labels = []
    image_count = len(images_pth)  
    for i,img_path in enumerate(images_pth):
        cl = img_path.split('/')[-2]
        class_index = cl
        train_labels.append(class_index)
        hist = []
        for dsc in desc:         
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            hist1 = dsc.describe(img[:,:,0])
            hist2 = dsc.describe(img[:,:,1])
            hist3 = dsc.describe(img[:,:,2])
            lbp_total = np.concatenate([hist1,hist2,hist3])
            maximum = calculate_max_value(numPoints)
            (hist_tmp, _) = np.histogram(lbp_total.ravel(),bins=np.arange(0, maximum + 3),range=(0, maximum + 2))
            hist.append(hist_tmp)
 
        hist = np.array(hist)
        hist = hist.reshape(-1)
        hist = hist / np.sum(hist)
        descriptor_list.append(hist)     
    descriptors =np.array(descriptor_list)
    train_labels = np.array(train_labels)
    return descriptors, train_labels
    

In [160]:
feat,lab = multiscale_lbp_extract_descriptors('../../../data/train', 8,train = True)    
test_feat, test_lab = multiscale_lbp_extract_descriptors('../../../data/valid',8,train = False)

In [145]:
"""from sklearn.neighbors import KNeighborsClassifier
acc,prec, rec, classif = train_model_and_calculate_accuracy(feat, lab, test_feat, test_lab)
import matplotlib.pyplot as plt
x1 = np.arange(1,51)
plt.plot(x1, acc, label = "accuracy",color = 'r')
plt.plot(x1,prec, label = 'precision')
plt.plot(x1, rec, label = 'recall',color = 'g')
plt.xlabel('number of neighbors')
plt.ylabel('percentage (%)')
plt.legend()
plt.grid()
plt.savefig('k-nearest_neighbor_total.jpg')"""

In [161]:
print('start svm')
svm, acc = search_for_best_rbf(feat, lab, test_feat, test_lab)
print('end svm')
#pred = svm.predict(t)

start svm
Best estimator found by grid search :  SVC(C=1000.0, gamma=1)
Accuracy score : 0.39643652561247217
end svm


In [ ]:
import pandas as pd
def evaluated_prediction(pred,test_lab,lab ):
    """
    Function which evaluate quality of prdiction of linear svm, calculating TP,FP,FN,TN

    Input pred = prediction
    Input test_lab = labels
    Input num_classes(15) = number of classes
    Input lab = name of classes

    Output  res = dataframe with all these values
    """
    num_classes = len(lab)
    tp = []
    fp = []
    fn = []
    tn = []
    for i in range(num_classes):
        tp_temp = 0
        fp_temp = 0
        fn_temp = 0
        tn_temp = 0
        for j in range(len(pred)):
            if(pred[j]==lab[i] and test_lab[j]==lab[i]):
                tp_temp = tp_temp + 1
            if(pred[j]==lab[i] and test_lab[j]!=lab[i]):
                fp_temp = fp_temp + 1
            if(pred[j]!=lab[i] and test_lab[j]==lab[i]):
                fn_temp = fn_temp + 1
            if(pred[j]!=lab[i] and test_lab[j]!=lab[i]):

                tn_temp = tn_temp +1
        tp.append(tp_temp)
        fp.append(fp_temp)
        fn.append(fn_temp)
        tn.append(tn_temp)
    data = {'labels':lab , 'True positive':tp,'True negative':tn,'False positive':fp,'False negative':fn}
    res = pd.DataFrame(data, columns = ['labels','True positive','True negative','False positive','False negative'])
    return res


def build_confusion_matrix(df,pred,test_labels,lab):
    """
    Function tu construct confusion matrix
    """
    num_classes = len(lab)
    cm = np.zeros((num_classes,num_classes))
    # insert true positive on the diagonal
    for i in range(num_classes):
        cm[i,i] = df.loc[i]['True positive']
    for i in range(num_classes): # lavoro sulle classes true
        for j in range(num_classes): #lavoro su classes predicted
            temp = 0
            for k in range(len(test_labels)):
                if(test_labels[k]==lab[i] and pred[k]==lab[j]):
                    temp = temp +1
            cm[i,j]=temp
    return cm


In [144]:
def train_model_and_calculate_accuracy(train_features,train_lab,test_features,test_lab): 
    scaler = StandardScaler()
    scaler.fit(train_features)

    X_train = scaler.transform(train_features)
    X_test = scaler.transform(test_features)
    clc = []
    acc = []
    prec = []
    rec = []
    for i in range(1,51):
        classifier = KNeighborsClassifier(n_neighbors=i) 
        classifier.fit(X_train, train_lab)
        clc.append(classifier)
        y_pred = classifier.predict(X_test)
        acc_temp = 0
        total = y_pred.shape[0]
        for j in range(y_pred.shape[0]):
            if y_pred[j]==test_lab[j]:
                acc_temp = acc_temp + 1
        precision, recall, fbeta, support = precision_recall_fscore_support(test_lab, y_pred)
        prec.append(np.mean(precision))
        rec.append(np.mean(recall))
        acc.append(acc_temp/total)
    return acc, prec, rec,clc 